In [ ]:
import tempfile

from monai.visualize.utils import matshow3d, blend_images
from monai.data import CacheDataset, ThreadDataLoader, PersistentDataset
from monai.transforms import (
    Compose,
    LoadImaged,
    MapTransform,
    ScaleIntensityRanged,
    Orientationd,
    Spacingd,
    CropForegroundd,
    SpatialCropd,
    RandFlipd,
    RandRotate90d,
    RandShiftIntensityd,
    EnsureTyped,
    BorderPad,
    SaveImaged,
    Lambdad,
)

from monai.data import  decollate_batch

import matplotlib.pyplot as plt
from monai.networks.nets import SwinUNETR

from monai.losses import DiceCELoss

from monai.metrics import DiceMetric, ROCAUCMetric

from monai.transforms import AsDiscrete

from tqdm import tqdm

from nodule_transforms.LoadArterysMaskd import LoadArterysMaskd
from nodule_transforms.LoadLIDCMaskd import LoadLIDCMaskd
from nodule_transforms.AddCustomerMetadatad import AddCustomerMetadatad
from nodule_transforms.CropToNoduled import CropToNoduled



import os
import torch



from data_tools.datatools import load_LUNA16_datalist


directory = './monai_data'
root_dir = tempfile.mkdtemp() if directory is None else directory
cache_dir = './training_cache'

PRE_PROCESSED_DIRECTORY = '/home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_luna16_full_range_1mm_malig/'

LUNA16_DATASET_DIR = '/media/vlermakov/data/LUNADataset'

datalist = load_LUNA16_datalist(LUNA16_DATASET_DIR,PRE_PROCESSED_DIRECTORY)

In [2]:
processed_scans = []
unprocessed_scans = []
n_processed = 0
for p in datalist:
    if p['pre_processed']:
        n_processed += 1 
        processed_scans.append(p)
    else:
        unprocessed_scans.append(p)
print(len(unprocessed_scans))

618


In [3]:
def file_name_formatter(metadict: dict, saver) -> dict:
    return {"subject": metadict['subset_id'] + "__" + metadict['series_id']+"_label_"+str(metadict['label'])+".extension", "idx": 0}
    
    
if(len(unprocessed_scans) > 0):
    pre_process_transforms = Compose(
        [
            LoadImaged(keys=["image"], ensure_channel_first=True, image_only=False),
            LoadLIDCMaskd(keys=["mask"], ensure_channel_first=True),
            AddCustomerMetadatad(keys=["image","mask"],meta_keys=["subset_id","series_id","label"]),
            ScaleIntensityRanged(
                keys=["image"],
                a_min=-1000,
                a_max=1000,
                b_min=0.0,
                b_max=1.0,
                clip=True,
            ),
            Spacingd(
                keys=["image","mask"],
                pixdim=(1.0, 1.0, 1.0),
                mode=("bilinear","nearest"),
            ),
            CropToNoduled(keys=["image","mask"]),
            SaveImaged(keys=["image"], output_name_formatter=file_name_formatter ,output_dir=PRE_PROCESSED_DIRECTORY, output_postfix="image", output_ext=".nii.gz", resample=False),
            SaveImaged(keys=["mask"], output_name_formatter=file_name_formatter, output_dir=PRE_PROCESSED_DIRECTORY, output_postfix="mask", output_ext=".nii.gz", resample=False)
        ]
    )
    
    train_ds = CacheDataset(
        data=unprocessed_scans,
        transform=pre_process_transforms,
        cache_num=len(unprocessed_scans),
        cache_rate=1.0,
        num_workers=8,
    )
    
#  d[key].meta['phonetic_id'] = d['phonetic_id']
#  d[key].meta['patch_id'] = d['patch_id']
#  d[key].meta['mrn'] = d['mrn']

Loading dataset:   0%|                                                                               | 0/618 [00:00<?, ?it/s]

Found annotations file ./LIDC-XML-only/tcia-lidc-xml/185/076.xml
(0) Large Nodule: 77.000006 - 78.000006
(0) Large Nodule: 79.000006 - 81.000006
(0) Large Nodule: 87.00000599999998 - 88.00000599999998
(1) Large Nodule: 78.000006 - 77.000006
(1) Large Nodule: 80.000006 - 80.000006
(2) Large Nodule: 81.000006 - 80.000006
(2) Large Nodule: 78.000006 - 77.000006
(3) Large Nodule: 78.000006 - 77.000006
(3) Large Nodule: 81.000006 - 79.000006
2023-10-06 20:21:49,691 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_luna16_full_range_1mm_malig/subset6__1.3.6.1.4.1.14519.5.2.1.6279.6001.129007566048223160327836686225_label_2.75/subset6__1.3.6.1.4.1.14519.5.2.1.6279.6001.129007566048223160327836686225_label_2.75_image_0.nii.gz
Found annotations file ./LIDC-XML-only/tcia-lidc-xml/189/045.xml
(1) Large Nodule: 92.9999968 - 89.9999968
(2) Large Nodule: 93.9999968 - 88.9999968
(2) Large Nodule: 72.00000000000001 - 63.000000000000014
(2) Larg

2023-10-06 20:23:00,924 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_luna16_full_range_1mm_malig/subset6__1.3.6.1.4.1.14519.5.2.1.6279.6001.315770913282450940389971401304_label_3.5/subset6__1.3.6.1.4.1.14519.5.2.1.6279.6001.315770913282450940389971401304_label_3.5_image_0.nii.gz
Found annotations file ./LIDC-XML-only/tcia-lidc-xml/185/086.xml
Found annotations file ./LIDC-XML-only/tcia-lidc-xml/185/131.xml
(0) Large Nodule: 208.0 - 211.0
(1) Large Nodule: 211.0 - 208.0
(2) Large Nodule: 211.0 - 208.0
(2) Large Nodule: 96.0 - 93.0
(3) Large Nodule: 212.0 - 207.0
(0) Large Nodule: 105.0 - 107.0
(3) Large Nodule: 96.0 - 92.0
(3) Large Nodule: 70.0 - 67.0
(1) Large Nodule: 107.0 - 105.0
(2) Large Nodule: 112.0 - 107.0
(2) Large Nodule: 107.0 - 105.0
(3) Large Nodule: 107.0 - 105.0
Found annotations file ./LIDC-XML-only/tcia-lidc-xml/185/212.xml
2023-10-06 20:23:07,143 INFO image_writer.py:197 - writing: /home/vlermakov/Developm

Loading dataset:   0%|                                                                   | 1/618 [01:52<19:19:10, 112.72s/it]

2023-10-06 20:23:19,951 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_luna16_full_range_1mm_malig/subset6__1.3.6.1.4.1.14519.5.2.1.6279.6001.254473943359963613733707320244_label_3.25/subset6__1.3.6.1.4.1.14519.5.2.1.6279.6001.254473943359963613733707320244_label_3.25_mask_0.nii.gz


...

Loading dataset:  98%|███████████████████████████████████████████████████████████████████▉ | 608/618 [43:16<00:45,  4.53s/it]

2023-10-06 21:04:48,133 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_luna16_full_range_1mm_malig/subset7__1.3.6.1.4.1.14519.5.2.1.6279.6001.223650122819238796121876338881_label_A/subset7__1.3.6.1.4.1.14519.5.2.1.6279.6001.223650122819238796121876338881_label_A_image_0.nii.gz
2023-10-06 21:04:48,710 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_luna16_full_range_1mm_malig/subset7__1.3.6.1.4.1.14519.5.2.1.6279.6001.223650122819238796121876338881_label_A/subset7__1.3.6.1.4.1.14519.5.2.1.6279.6001.223650122819238796121876338881_label_A_mask_0.nii.gz
2023-10-06 21:04:50,616 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_luna16_full_range_1mm_malig/subset7__1.3.6.1.4.1.14519.5.2.1.6279.6001.271307051432838466826189754230_label_4.25/subset7__1.3.6.1.4.1.14519.5.2.1.6279.6001.271307051432838466826189754230_labe

Loading dataset:  99%|████████████████████████████████████████████████████████████████████ | 610/618 [43:29<00:39,  4.98s/it]

2023-10-06 21:04:56,944 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_luna16_full_range_1mm_malig/subset7__1.3.6.1.4.1.14519.5.2.1.6279.6001.129982010889624423230394257528_label_1.25/subset7__1.3.6.1.4.1.14519.5.2.1.6279.6001.129982010889624423230394257528_label_1.25_image_0.nii.gz
2023-10-06 21:04:57,035 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_luna16_full_range_1mm_malig/subset7__1.3.6.1.4.1.14519.5.2.1.6279.6001.129982010889624423230394257528_label_1.25/subset7__1.3.6.1.4.1.14519.5.2.1.6279.6001.129982010889624423230394257528_label_1.25_mask_0.nii.gz


Loading dataset: 100%|█████████████████████████████████████████████████████████████████████| 618/618 [43:29<00:00,  4.22s/it]


In [ ]:
img = train_ds[1]['image']
mask = train_ds[1]['mask']
print(f"Label: {train_ds[1]['label']}")

blended_image = blend_images(img, mask, alpha=0.5, cmap="hsv")
# Generate two plot windows with two figures
fig1 = plt.figure()

matshow3d(blended_image, channel_dim = 0, fig = fig1, every_n=5, frame_dim=-1);#, cmap='gray');

In [ ]:
datalist

In [ ]:
mhd_file = '/media/vlermakov/data/LUNADataset/subset2/1.3.6.1.4.1.14519.5.2.1.6279.6001.235364978775280910367690540811.mhd'

print(os.path.basename(mhd_file))

In [ ]:
series_id = os.path.basename(mhd_file).rsplit(".", 1)[0]
subset_id = os.path.basename(os.path.dirname(mhd_file))

In [ ]:
series_id

In [ ]:
subset_id

In [ ]:
def folder_exists_with_partial_name(search_path, partial_name):
    """
    Checks if a folder starting with the given partial_name exists within the search_path.
    
    :param search_path: The path to search within.
    :param partial_name: The starting name of the folder to search for.
    :return: True if a matching folder is found, else False.
    """
    
    for folder_name in os.listdir(search_path):
        if folder_name.startswith(partial_name) and os.path.isdir(os.path.join(search_path, folder_name)):
            return True
    return False

In [ ]:
subset_id = 'subset2'
series_id = '1.3.6.1.4.1.14519.5.2.1.6279.6001.311236942972970815890902714604'

folder_partial_name = os.path.join(PRE_PROCESSED_DIRECTORY, subset_id + "__" + series_id)
print(folder_exists_with_partial_name(PRE_PROCESSED_DIRECTORY, subset_id + "__" + series_id))